# lawsuit related to IPO

In [1]:
import pandas as pd

In [3]:
# Obtain the csv file in the target file
file_dir = r"D:/polyu/data_polyu/law_suit_data/"

raw_data = pd.read_csv(file_dir + "matching_data.csv")
raw_data = raw_data[raw_data["ipo_date"].notna()]


In [3]:
# obtained from WRDS CRSP/Compustat Merged Database and provides "gvkey", "PERMNO" and "ipo date"
match_data = pd.read_csv(file_dir +  "matching.csv", usecols=["LPERMNO", "gvkey", "tic", "LINKENDDT", "ipodate"])
match_data.rename(columns={"tic":"Ticker"}, inplace=True)

# take the Ticker with newest date: LINKENDDT
match_data["Max_LINKENDDT"] = match_data.groupby("Ticker")["LINKENDDT"].transform("max")
match_data = match_data[match_data["Max_LINKENDDT"] == match_data["LINKENDDT"]]
match_data = match_data.drop(columns=["Max_LINKENDDT", "LINKENDDT"])

In [4]:
# provides "PERMNO" and its corresponding using period: "NAMEDT","NAMEENDT". 
full_data = pd.read_sas(file_dir + "msenames.sas7bdat")
full_data = full_data[["NAMEDT", "NAMEENDT", "TICKER", "PERMNO"]].drop_duplicates()
full_data.rename(columns={"TICKER": "Ticker"}, inplace=True)

# take the Ticker with newest date: LINKENDDT
full_data['MAX_NAMEENDT'] = full_data.groupby(['Ticker', 'PERMNO'])['NAMEENDT'].transform('max')
full_data = full_data[full_data['NAMEENDT'] == full_data['MAX_NAMEENDT']]
full_data.drop(columns=['MAX_NAMEENDT'], inplace=True)


In [5]:
# transfer the data type from "bytes string" to "string"
def bytes_to_string(b):
    return b.decode('utf-8') if isinstance(b, bytes) else b

ipo_data = ipo_data.applymap(bytes_to_string)
full_data = full_data.applymap(bytes_to_string)

In [6]:
# print(len(raw_data[raw_data['Filing Name'].str.contains(':')]))
exclude_exchanges = ["Open-end Fund", "Privately Traded", "Pink Sheets", "ETF"]
# print(len(raw_data[raw_data['Exchange'].isin(exclude_exchanges)]))
raw_data = raw_data[~raw_data['Filing Name'].str.contains(':')]
raw_data = raw_data[~raw_data['Exchange'].isin(exclude_exchanges)]
raw_data = raw_data.dropna(subset=["Ticker"])

In [7]:
def get_ipo(raw_data, data, column_need):
    data = data[["Trade_Date", column_need, "gvkey"]]
    data["Max_Trade_Date"] = data.groupby(column_need)["Trade_Date"].transform("max")
    data = data[data["Trade_Date"] == data["Max_Trade_Date"]]
    data = data.drop(columns=["Max_Trade_Date"])
    
    if column_need != "Ticker":
        data.rename(columns={column_need: "Ticker"}, inplace=True)
    matched_data = pd.merge(raw_data, data, on=["Ticker"], how="left")
    return matched_data

aa = get_ipo(raw_data, ipo_data, "Ticker")
aa = get_ipo(aa, ipo_data, "tic")
aa["gvkey"] = aa["gvkey_x"].combine_first(aa["gvkey_y"])
aa = aa.drop(columns=["gvkey_x", "gvkey_y", "Trade_Date_x"])
aa["Filing Date"] = pd.to_datetime(aa["Filing Date"])

In [ ]:
dd.to_csv(file_dir + "matching_data.csv")

In [39]:
len(bb["ipo_date"].unique())

483